UPDATE: 2018-10-13 Sat. 

 -*- coding: utf-8 -*-pack
 @Initial wrote Date    : 2018-09-30 15:56:57
 @Author  : Jinyi Kuang (jkuang000@gmail.com)
 @Link    : https://github.com/jinyikuang000

#INTRO------------------------------------------------

 This .py file automate the canvas assignment grading
 with seleium and chromedriver.
 It requires phone to enter two step verification code.
 The script can be modified to grade different hw.

#-----------------------------------------------------




## 1 set up & log in

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-pack
# @Date    : 2018-09-30 15:56:57
# @Author  : Jinyi Kuang (jkuang000@gmail.com)
# @Link    : https://github.com/jinyikuang000

# INTRO------------------------------------------------
#
# This .py file automate the canvas assignment grading
# with seleium and chromedriver.
# It requires phone to enter two step verification code.
# The script can be modified to grade different hw.
# 
#
#-----------------------------------------------------


# LOAD MODULES-----------------------------------------------------

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import ui
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException


import os
import time
import unittest  # python package to write test
import re




# SET UP DRIVER-----------------------------------------------------

# paste geckodrive in the path /usr/local/bin
driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver')

# set window size
driver.set_window_size(1341, 810)

# change window location to the left side of  the screen
driver.set_window_position(20,0)

# maximize window
#driver.maximize_window()


# INITIALTION ------------------------------------------------
#
# element_has_value(object) used in checking if gradebox is empty
#
#-----------------------------------------------------
# custom wait condition - def function
class element_has_value(object):
  """
  An expectation for checking that an element has a value.
  the object have the following attributes
  locator - used to find the element
  returns the element if it has non-NULL value
  """
  def __init__(self,locator):
    self.locator = locator
    

  def __call__(self, driver):
    element = driver.find_element(*self.locator)   # Finding the referenced element
    if element.get_attribute("value")!="--":
        return element
    else:
        return False   

# LOG IN-----------------------------------------------------

driver.get("https://canvas.upenn.edu/")

# Select the id box
username_box = driver.find_element_by_name('login')

# Send id information
username_box.send_keys('jkuang')

# Find password box
password_box = driver.find_element_by_name('password')

# Send password
password = input("Please enter your password:")
password_box.send_keys(password)

# find  lgin button
login_button = driver.find_element_by_name('submit1')

# Click login
login_button.click()

trust_checkbox = driver.find_element_by_name('trustUA')
trust_checkbox.click()

# name the canvas window as main_window
main_window = driver.current_window_handle

#-----------------------------------------------------
# Open a new window

# This does not change focus to the new window for the driver.
driver.execute_script("window.open('');")

# Switch to the new window
driver.switch_to.window(driver.window_handles[1])

# enter link to 2 step verification
driver.get("https://twostep.apps.upenn.edu/twoFactor/twoFactorPublicUi/app/UiMainPublic.index")

# wait for 2 seconds
time.sleep(2)

# find the nested hidden element of sendtext button and click
sendtext_button = driver.find_element_by_xpath("//form[input/@name='OWASP_CSRFTOKEN']")
sendtext_button.click()

# Close current tab
time.sleep(1)
driver.close()

# Put focus on current window which will be the window opener
driver.switch_to_window(main_window)

#-----------------------------------------------------
# send code to phone
twostep = driver.find_element_by_id('penntoken')
twostep.click()

# enter the code mannually
# wait for at most 30 seconds untill text is being entered
WebDriverWait(driver, 30).until(
     EC.presence_of_element_located((By.CLASS_NAME, "ic-DashboardCard__link"))
 ) 

#-----------------------------------------------------
driver.switch_to_window(main_window)
# find PPE 313
PPE313 = driver.find_element_by_class_name('ic-DashboardCard__link')
PPE313.click()

#-----------------------------------------------------
# CHOOSE ASSIGNMENT TO GRADE
#-----------------------------------------------------
# Wait until the page loaded (up to 10 seconds)
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "flash_screenreader_holder"))
) 

assignment = driver.find_element_by_link_text('Assignment #4')
assignment.click()

#-----------------------------------------------------
driver.switch_to.window(main_window)
speedgrader = driver.find_element_by_class_name('icon-speed-grader')
speedgrader.click()

#-----------------------------------------------------
# FIND SPECIFIC STUDENT TO START
#-----------------------------------------------------
driver.switch_to.window(driver.window_handles[1])

WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.ID, "speedgrader_iframe"))
) 
# # Find specific student
# test_student = driver.find_element_by_xpath("//div/select/option[@value='5529130']")
# test_student.click()



## 2  Grade question 6 & 12 by students 

In [ ]:
# -----------------------------------------------------
# STEP 2
# GRADE BY STUDENTS
#
# workflow: 
# grade all questions for each students
# loop break untill all missing submission
#
#-----------------------------------------------------


driver.switch_to.window(driver.window_handles[1])

student_index = driver.find_element_by_id('x_of_x_students_frd')

while student_index.text !='176/176':


    #-----------------------------------------------------
    # PREPARE TO GRADING 
    #-----------------------------------------------------

    driver.switch_to.window(driver.window_handles[1])

    # Wait until the page loaded (up to 10 seconds)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "speedgrader_iframe"))) 
    
    # element is inbedded in frame [<iframe> tag specifies an inline frame.]
    driver.switch_to.frame(driver.find_element_by_id("speedgrader_iframe"))

    #-----------------------------------------------------
    # START GRADING Q6 & Q13
    #-----------------------------------------------------

    # QUESTION 6 ---------------------------------------------------

    # find gradebox -> click -> clear

    q1= driver.find_element_by_name('question_score_46372089')
    q1.clear()
    driver.execute_script("window.scrollBy(0, 250);")
    q1.click()
    
    point1 = input("Points for Q6:")
    q1.send_keys(point1)
    

    # Wait until an element value is not empty
    WebDriverWait(driver, 30).until(element_has_value((By.NAME, 'question_score_46372089')))
    


    # QUESTION 13 ---------------------------------------------------

    # find grade box -> click -> clear
    driver.execute_script("arguments[0].scrollIntoView();", driver.find_element_by_id("question_46372115_question_text"))

    q3 = driver.find_element_by_name('question_score_46372115')
    q3.clear()
    driver.execute_script("window.scrollBy(0, 100);")
    q3.click()
    

    point3 = input("Points for Q13:")
    q3.send_keys(point3)
    


    # wait untill grades being entered then move to next question
    WebDriverWait(driver, 30).until(element_has_value((By.NAME, 'question_score_46372115')))
    
    #-----------------------------------------------------
    # UPDATE SCORE 
    #-----------------------------------------------------
    # driver.switch_to.window(driver.window_handles[1])

    # driver.switch_to.frame(driver.find_element_by_id("speedgrader_iframe"))

    # click  update score
    update_score_button = driver.find_element_by_xpath("//div[@id='update_scores']/div/div/button[@class='btn btn-primary update-scores']")
    update_score_button.click()
    time.sleep(1)

    #-----------------------------------------------------
    # PRINT SCORE FOR RECORD 
    #-----------------------------------------------------
    driver.switch_to.window(driver.window_handles[1])

    # find student name
    student_name = driver.find_element_by_xpath("//span[@class='ui-selectmenu-item-header']")
    
    print(student_name.text, " final score is ", sep="")

    # find score
    driver.switch_to.frame(driver.find_element_by_id("speedgrader_iframe"))
    score = driver.find_element_by_id('after_fudge_points_total')
    
    print(score.text)

    driver.switch_to.default_content()
    
    # wait for 2 seconds to check if the record is correct  
    # time.sleep(2)

    #-----------------------------------------------------
    # NEXT STUDENT
    #-----------------------------------------------------

    driver.switch_to.window(driver.window_handles[1])

    # Wait until the page loaded (up to 10 seconds)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "next-student-button")))

    # find next student arrow
    next_arrow = driver.find_element_by_xpath("//button[@id='next-student-button']/i[contains(@class,'icon-arrow-right next')]")

    # click with JS
    driver.execute_script("$(arguments[0]).click();", next_arrow)

    # update student_index 
    driver.switch_to.window(driver.window_handles[1])

    student_index = driver.find_element_by_id('x_of_x_students_frd')

    ##########################################################

    if student_index.text !='176/176':
        print(student_index.text)

    else:

        print("FINISHED GRADING and SWITCH BACK TO MAIN WINDOW")
        driver.switch_to_window(main_window)



## REVISE QUESTION 13 
note:
Selenium provides a convenient Select class to work with select -> option constructs:
https://stackoverflow.com/questions/7867537/selenium-python-drop-down-menu-option-value

The student list was exported to an excel file from url: https://canvas.upenn.edu/courses/1414109/quizzes/2061372/statistics. Then select Student Analysis

In [ ]:
# the namelist and value list can be found in dropbox
valuelist = [
'5282800',
'5279873',
'5193746',
'5281438',
'5193866',
'3772710',
'5281120',
'5195196',
'5293933',
'5280101',
'5281093',
'5285632',
'5280490',
'5280374',
'5192663',
'5389304',
'5195170',
'5199352',
'5120764',
'5193252',
'5124585',
'5281149',
'5488236',
'5488233',
'5192969',
'5193080',
'5280096',
'5382272',
'5195024',
'5195200']


In [ ]:
#-----------------------------------------------------
# REVISE QUESTION 14 
#-----------------------------------------------------

driver.switch_to.window(driver.window_handles[1])

# get the select element whcih contains a all students' names
select = Select(driver.find_element_by_id('students_selectmenu'))


for student in valuelist:
    
    select = Select(driver.find_element_by_id('students_selectmenu'))
    target_student = select.select_by_value(student)
    driver.execute_script("$(arguments[0]).click();", target_student )
    
        
    #-----------------------------------------------------
    # REVISING SCORE FOR QUESTION 14
    #-----------------------------------------------------
        
    driver.switch_to.window(driver.window_handles[1])
    
    driver.switch_to.frame(driver.find_element_by_id("speedgrader_iframe"))

    
    # find grade box -> click -> clear
    driver.execute_script("arguments[0].scrollIntoView();", driver.find_element_by_id("question_46372107_question_text"))
    
    q14 = driver.find_element_by_name('question_score_46372107')
#    q14.clear()
    driver.execute_script("window.scrollBy(0, 250);")
    q14.click()
        
    
    point14 = input("Points for Q14:")
    q14.send_keys(point14)
        
    
    
    # wait untill grades being entered then move to next question
    WebDriverWait(driver, 60).until(element_has_value((By.NAME, 'question_score_46372107')))
    
    #-----------------------------------------------------
    # UPDATE SCORE 
    #-----------------------------------------------------
    # driver.switch_to.window(driver.window_handles[1])

    # driver.switch_to.frame(driver.find_element_by_id("speedgrader_iframe"))

    # click  update score
    update_score_button = driver.find_element_by_xpath("//div[@id='update_scores']/div/div/button[@class='btn btn-primary update-scores']")
    update_score_button.click()
    time.sleep(1)
    
    #-----------------------------------------------------
    # PRINT NAME AND SCORE
    #-----------------------------------------------------
    driver.switch_to.window(driver.window_handles[1])

    # find student name
    student_name = driver.find_element_by_xpath("//span[@class='ui-selectmenu-item-header']")
    
    print(student_name.text, " final score is ", sep="")

    # find score
    driver.switch_to.frame(driver.find_element_by_id("speedgrader_iframe"))
    score = driver.find_element_by_id('after_fudge_points_total')
    
    print(score.text)

    driver.switch_to.default_content()
    